In [12]:
# text file pre-processing function

import re

def preprocess_pdf_text(text):
    # Step 1: Remove page numbers (assuming they appear on a line by themselves)
    text = re.sub(r'\n\d+\n', '\n', text)

    # Step 2: Remove common headers and footers (heuristic)
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        if len(line.strip()) < 3:  # Skip short lines (e.g., single letters or digits)
            continue
        if re.match(r'^(Page|PAGE)\s*\d+', line.strip()):  # Page indicators
            continue
        cleaned_lines.append(line)
    text = '\n'.join(cleaned_lines)

    # Step 3: Fix hyphenated line breaks (e.g., "inter-\nview" → "interview")
    text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)

    # Step 4: Merge lines that are artificially split (end of line not a sentence end)
    # First, normalize line endings
    text = re.sub(r'\r\n?', '\n', text)

    # Then merge lines that are not paragraph breaks
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)  # convert single line breaks to spaces

    # Step 5: Normalize multiple newlines (paragraph breaks)
    text = re.sub(r'\n{2,}', '\n\n', text)

    # Step 6: Normalize whitespace
    text = re.sub(r'[ \t]+', ' ', text)  # collapse tabs and spaces
    text = re.sub(r' +\n', '\n', text)   # trim trailing spaces on lines
    text = text.strip()

    return text


In [13]:
# chunking the document function

import tiktoken

def chunk_text(text, chunk_size=100, overlap=15):
    encoding = tiktoken.encoding_for_model("text-embedding-3-large")
    tokens = encoding.encode(text)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        decoded = encoding.decode(chunk)
        chunks.append(decoded)
    return chunks


In [14]:
# category definitions

categories = {
    "Clinical Decision Support": (
        "Artificial intelligence systems that assist healthcare providers in clinical decision-making by analyzing patient data to generate diagnostic suggestions, treatment recommendations, and personalized care plans. These tools may use rules-based logic, machine learning, or probabilistic reasoning to help clinicians make timely and evidence-based decisions, especially in complex or high-risk scenarios."
    ),

    "Clinical Documentation AI": (
        "AI applications that streamline or automate the creation, management, or structuring of clinical documentation. This includes generating summary reports, transcribing and organizing clinical notes, auto-completing documentation during consultations, and extracting structured data from free-text clinical narratives. These systems reduce administrative burden, improve documentation quality, and integrate with electronic health records (EHRs)."
    ),

    "Medical Imaging AI": (
        "AI tools used to interpret, analyze, or enhance medical imaging data such as radiology scans, pathology slides, and ophthalmology images. These systems may perform tasks like detecting abnormalities (e.g., tumors, fractures), segmenting anatomical structures, quantifying lesions, or prioritizing imaging workflows. Techniques include convolutional neural networks (CNNs), image classification, and computer vision-based diagnostics."
    ),

    "Predictive Analytics": (
        "AI models that analyze historical and real-time clinical data to forecast future patient outcomes. These models are used for risk stratification, early warning systems, prediction of readmission or mortality, and estimating treatment effectiveness. Common data inputs include vitals, labs, medications, demographics, and medical history. Techniques may include regression, time-series analysis, and deep learning."
    ),

    "Operational and Administrative Automation": (
        "AI systems designed to optimize hospital operations and administrative workflows. This includes automating scheduling, resource allocation, billing, claims management, supply chain logistics, and staff workflow optimization. These systems improve efficiency, reduce human error, and enhance the management of hospital throughput and resources using algorithms and machine learning."
    ),

    "Patient-facing AI": (
        "AI applications that interact directly with patients to support health monitoring, self-care, education, or triage. These include chatbots and virtual assistants, symptom checkers, wearable device integrations, and remote monitoring platforms. They provide personalized guidance, collect patient-reported outcomes, and may flag concerning symptoms for provider review."
    ),

    "Robotics and Surgical AI": (
        "AI systems integrated into robotic platforms used in surgical or interventional procedures. These include robotic-assisted surgery systems for enhanced precision, AI-guided tools for minimally invasive techniques, and rehabilitation robotics used post-operatively. These systems combine real-time sensing, motion control, and decision support to improve surgical outcomes and reduce human error."
    ),

    "Education and Training AI": (
        "AI tools used in healthcare education and workforce development. This includes virtual patient simulations, intelligent tutoring systems, curriculum personalization tools, and performance analytics for medical trainees. These systems aim to enhance clinical reasoning skills, procedural knowledge, and continuing education using adaptive learning and natural language processing."
    ),

    "Research and Clinical Trial AI": (
        "AI platforms that support biomedical research and clinical trial operations. These tools assist with patient cohort identification, trial design optimization, natural language processing of scientific literature, data analysis, and drug discovery. They help accelerate evidence generation, hypothesis testing, and treatment development in both pre-clinical and clinical phases."
    ),

    "Public Health AI": (
        "AI systems used to analyze population-level health data to support public health policy and intervention. Applications include epidemiological modeling, outbreak detection, surveillance of disease trends, analysis of social determinants of health, and health equity analysis. These tools support proactive, data-driven public health strategies and real-time monitoring."
    )
}



In [15]:
# category examples

category_examples = {
    "Clinical Decision Support": [
        "Once validated, its use can be envisioned in a wide range of scenarios, including decision support in existing practice.",
        "Transparently disclose the use of GenAI and its role in decision making.",
        "AI suggests treatment plans for a patient based on medical history.",
        "Computerized clinical decision support systems (CCDSSs) provide patient-specific assessments or recommendations to clinicians to aid clinical decision making.",
        "Some systems suggest diagnostic tests, while others suggest treatments or preventative measures, but these should be carefully evaluated by the healthcare professional before usage."
    ],

    "Clinical Documentation AI": [
        "Electronic health record systems should integrate artificial intelligence to increase efficiency in simple documentation, allowing healthcare professionals to spend more time attending to patient needs.",
        "Generative AI usage for patient visit summaries should be approached with caution, due to risks of inaccurate information communication.",
        "GenAI-enabled clinical documentation tools assist in creating encounter summaries, generating draft notes, and extracting structured data from conversations."
    ],

    "Medical Imaging AI": [
        "Recently a transformational advance in automated retinal image analysis, using Deep Learning algorithms, has been demonstrated.",
        "Some CCDSSs include radiographic image interpretation tools that help clinicians identify abnormalities.",
        "Experts recommend using AI-assisted image analysis systems as a first consultation, deferring back to the radiologist for confirmation.",
        "Although traditional ML-based approaches dominate imaging, emerging GenAI models show promise in tasks such as image synthesis and labeling."
    ],

    "Predictive Analytics": [
        "A model trained to predict the likelihood of death from pneumonia assigned lower risk to patients with asthma, but only because such patients were treated as higher priority by the hospital.",
        "CCDSSs may calculate risk scores for conditions such as cardiovascular disease or osteoporosis, helping identify high-risk patients.",
        "Risk prediction tools integrate patient data to estimate the probability of future adverse health events."
    ],

    "Operational and Administrative Automation": [
        "AI software has been integrated into the organizational infrastructure of our hospital operations.",
        "Using AI, hospital workers' shift scheduling can be automated to reduce secretarial burdens.",
        "GenAI applications in administrative tasks include automating prior authorizations and summarizing payer policies."
    ],

    "Patient-facing AI": [
        "Kardia Mobile uses a finger pad and a smartphone app to record an EKG. The platform claims to use AI-enabled detection of atrial fibrillation.",
        "CloudUPDRS, an AI algorithm differentiates between actual tremors and bad data, enabling Parkinson's patients to perform in-home testing.",
        "There is a proliferation of companies developing apps that offer online doctors' appointments. Babylon claims to use an AI algorithm to automatically triage patients virtually.",
        "Chatbots powered by generative AI can respond to patient queries, schedule appointments, and assist in medication adherence.",
        "Concerns regarding the lack of verification in patient-facing AI tools suggest mitigation of such software in high-stakes situations."
    ],

    "Robotics and Surgical AI": [
        "AI-integrated surgical machines can help increase the precision in difficult medical procedures.",
        "In post-op physical therapy sessions, machines integrated with a patient's progress and data can help provide the appropriate training.",
        "If the AI pacemaker fails, the responsibility to which this falls upon remains ambiguous."
    ],

    "Education and Training AI": [
        "All work completed by medical students should be conducted without the usage of artifical intelligence.",
        "GenAI has potential in medical training, for example, simulating patient interactions or generating test questions tailored to learning goals.",
        "Hospitals should integrate AI into staff trainings because it provides a customized experience."
    ],

    "Research and Clinical Trial AI": [
        "We need your help to take personalized medicine to its full potential, and develop a Machine Learning algorithm that automatically classifies genetic variations.",
        "It may support clinical trial design by generating hypotheses, summarizing relevant literature, or analyzing eligibility criteria from EHR data.",
        "Using artificial intelligence in research raises concerns about fake data, which can have lasting implications on medical practices if not verified."
    ],

    "Public Health AI": [
        "Support the development of wearable devices for the sensing of environmental toxins and broad-based pathogen sensing for rural and urban environments. The collected data can inform policies for mitigating risks of pandemics and epidemics.",
        "Providing access to data captured by mHealth apps and devices could enhance the research community's ability to build more insights into public health through AI.",
        "GenAI could aid public health monitoring by extracting trends from social media, news sources, or reports, enhancing situational awareness."
    ]
}


In [16]:
# Azure OpenAI setup

from openai import OpenAI, AzureOpenAI
import json
import pandas as pd
import regex as re
from dotenv import load_dotenv
import os

load_dotenv()  # Load from .env file

AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT_embeddings")
AZURE_API_KEY = os.getenv("AZURE_API_KEY_embeddings")
AZURE_API_VERSION = "2025-04-01-preview"

openai = AzureOpenAI(
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AZURE_API_KEY,
    api_version=AZURE_API_VERSION
)

In [17]:
# embedding function

def get_embedding(text: str, model="embedding3large"):
    response = openai.embeddings.create(
        model=model,
        input=text
    )
    return response.data[0].embedding


In [18]:
# create category def+ex embeddings (so output is just one embedding per category)
# just do one time

category_all_embeddings = {}

for label in categories:
    combined_text = categories[label] + " " + " ".join(category_examples.get(label, []))
    category_all_embeddings[label] = get_embedding(combined_text)

In [19]:
# cosine similarity and comparison functions

import numpy as np
from collections import defaultdict

def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def classify_with_combined_embeddings(chunk_embeddings, category_all_embeddings, threshold):
    classification = defaultdict(list)

    for idx, text, chunk_embedding in chunk_embeddings:
        for category, cat_embedding in category_all_embeddings.items():
            sim = cosine_similarity(chunk_embedding, cat_embedding)
            if sim >= threshold:
                classification[category].append((idx, sim, text))
    return classification


In [20]:
# Compute document-level similarity function

def compute_document_level_similarity(results, category_all_embeddings, threshold=0.6):
    document_scores = {}

    for category, matches in results.items():
        if matches:
            # Average cosine similarity of all matched chunks in this category
            sims = [sim for _, sim, _ in matches]
            avg_sim = sum(sims) / len(sims)
            document_scores[category] = avg_sim
        else:
            document_scores[category] = 0.0  # No matches → 0 similarity

    # Filter categories above threshold
    classified_categories = {
        category: score
        for category, score in document_scores.items()
        if score >= threshold
    }

    return document_scores, classified_categories


In [21]:
import pandas as pd
import os
import tempfile
import requests
from error_handling import save_failed_docs

# Extract text from PDF (returns text instead of saving to file)
def pdf_to_text(pdf_path):
    import fitz  # PyMuPDF
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    doc.close()
    return full_text

# Download PDF if link is a URL, with User-Agent header and error handling
def get_pdf_path(link):
    if link.startswith("http"):
        headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
        try:
            response = requests.get(link, headers=headers, timeout=15)
            response.raise_for_status()  # Raises HTTPError for bad responses
            tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
            tmp.write(response.content)
            tmp.close()
            return tmp.name
        except requests.exceptions.RequestException as e:
            print(f"[Download Error] Could not download {link}: {e}")
            return None
    else:
        return link  # assume local path

# Read CSV
csv_path = "/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/FINAL_dataset_10.csv"
df = pd.read_csv(csv_path)
results_list = []
failed_links = []

for idx, row in df.iterrows():
    pdf_link = row["URL"]
    pdf_path = get_pdf_path(pdf_link)
    if not pdf_path or not os.path.exists(pdf_path):
        print(f"[Skip] PDF not found or unreadable: {pdf_link}")
        failed_links.append(pdf_link)
        continue

    try:
        # Extract and preprocess text
        raw_text = pdf_to_text(pdf_path)
        clean_text = preprocess_pdf_text(raw_text)
        chunks = chunk_text(clean_text)

        # Embedding and classification
        chunk_embeddings = [(i, chunk, get_embedding(chunk)) for i, chunk in enumerate(chunks)]
        results = classify_with_combined_embeddings(chunk_embeddings, category_all_embeddings, threshold=0.6)
        document_scores, classified_categories = compute_document_level_similarity(results, category_all_embeddings, threshold=0.6)

        # Store results
        results_list.append({
            "URL": pdf_link,
            "AI Applications": list(classified_categories.keys()),
            "Similarity Scores": document_scores
        })

    except Exception as e:
        print(f"[Processing Error] Could not process {pdf_link}: {e}")
        failed_links.append(pdf_link)

    finally:
        # Clean up temp file if downloaded
        if pdf_link.startswith("http") and pdf_path and os.path.exists(pdf_path):
            os.remove(pdf_path)

# Save results to DataFrame/CSV
results_df = pd.DataFrame(results_list)
results_df.to_csv("AI_appl_results_10.csv", index=False)
print("✅ Batch classification complete. Results saved to AI_appl_results_10.csv")

# Save failed links at the end
save_failed_docs(
    failed_links,
    output_dir="/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type",
    filename="AI_appl_failed_docs_10.csv"
)


[Download Error] Could not download https://prod.drupal.gaotest.org/assets/gao-21-7sp.pdf: 403 Client Error: Forbidden for url: https://www.gao.gov/assets/gao-21-7sp.pdf
[Skip] PDF not found or unreadable: https://prod.drupal.gaotest.org/assets/gao-21-7sp.pdf
[Download Error] Could not download https://jtc1info.org/wp-content/uploads/2023/12/Overview_of_ISO_IEC_workshop_Wael.pdf: HTTPSConnectionPool(host='jtc1info.org', port=443): Max retries exceeded with url: /wp-content/uploads/2023/12/Overview_of_ISO_IEC_workshop_Wael.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe771220be0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
[Skip] PDF not found or unreadable: https://jtc1info.org/wp-content/uploads/2023/12/Overview_of_ISO_IEC_workshop_Wael.pdf
[Download Error] Could not download https://www.medrxiv.org/content/10.1101/2024.10.23.24315991v1.full.pdf: HTTPSConnectionPool(host='www.medrxiv.org', port

In [12]:
# Convert CSV to JSON

import csv
import json

def csv_to_json(csv_file_path, json_file_path):
    # Read the CSV and add data to a dictionary
    with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        reader = csv.DictReader(csv_file)
        data = list(reader)

    # Write the JSON file
    with open(json_file_path, mode='w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4)

# Example usage
csv_to_json('/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type/batch_classification4_results.csv', '/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type/batch_classification4_results.json')


In [22]:
import os
import pandas as pd
import tempfile
import requests
from error_handling import save_failed_docs


# Extract text from PDF (returns text instead of saving to file)
def pdf_to_text(pdf_path):
    import fitz  # PyMuPDF
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    doc.close()
    return full_text

# List of local PDF paths (update this list with your actual file paths)
pdf_folder = "/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type/prev_failed_docs"  # <-- update this path
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]

results_list = []
failed_links = []

for pdf_path in pdf_files:
    pdf_link = pdf_path  # Use the local path as the 'URL' field
    if not os.path.exists(pdf_path):
        print(f"[Skip] PDF not found: {pdf_link}")
        failed_links.append(pdf_link)
        continue
    try:
        # Extract and preprocess text
        raw_text = pdf_to_text(pdf_path)
        clean_text = preprocess_pdf_text(raw_text)
        chunks = chunk_text(clean_text)

        # Embedding and classification
        chunk_embeddings = [(i, chunk, get_embedding(chunk)) for i, chunk in enumerate(chunks)]
        results = classify_with_combined_embeddings(chunk_embeddings, category_all_embeddings, threshold=0.6)
        document_scores, classified_categories = compute_document_level_similarity(results, category_all_embeddings, threshold=0.6)

        # Store results
        results_list.append({
            "URL": pdf_link,
            "AI Applications": list(classified_categories.keys()),
            "Similarity Scores": document_scores
        })
    except Exception as e:
        print(f"[Processing Error] Could not process {pdf_link}: {e}")
        failed_links.append(pdf_link)

# Save results to DataFrame/CSV
results_df = pd.DataFrame(results_list)
results_df.to_csv("AI_appl_results_12_for_prev_failed.csv", index=False)
print("✅ Batch classification complete. Results saved to AI_appl_results_12_for_prev_failed.csv")

# Save failed links at the end
save_failed_docs(
    failed_links,
    output_dir="/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type",
    filename="AI_appl_failed_docs_12.csv"
)


✅ Batch classification complete. Results saved to AI_appl_results_12_for_prev_failed.csv
No failed documents to save.
